In [1]:
#libraries
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype
import os
import csv
import dataframe_image as dfi
import statistics

#pandas options
pd.set_option('display.max_columns', None)

# Create folder for file exports

In [2]:
#Create directory folder saving exports
outdir = './exports'
if not os.path.exists(outdir):
    os.mkdir(outdir)

# Load Data

In [3]:
#Load all medications data
df = pd.read_csv("dataset.csv")

In [4]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,custom1,custom2,custom3,SepalLengthCm.1,SepalWidthCm.1,PetalLengthCm.1,PetalWidthCm.1,Species.1
0,1,5.1,3.5,1.4,0.2,Iris-setosa,NaN,1,6,5.1,3.5,NaN,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa,NaN,1,6,4.9,3.0,NaN,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa,NaN,1,6,4.7,3.2,NaN,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa,NaN,1,6,NaN,3.1,NaN,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa,NaN,1,6,5.0,3.6,NaN,0.2,Iris-setosa


# Get full null columns and non-varying value columns

### Identify full null columns

In [5]:
#Identify columns where all values are NULL
full_null_cols_df = pd.DataFrame(df.isnull().all()).reset_index().rename(columns={'index': 'field name'})
full_null_cols_df = full_null_cols_df[full_null_cols_df[0] ==True]

#Export list of full null columns
fullname = os.path.join(outdir, 'full_null_cols_list.csv')
full_null_cols_df['field name'].to_csv(fullname, index=False)

### Identify columns where all values are the same

In [6]:
#Identify columns where all values are the same
num_unique_vals_df = pd.DataFrame(df.nunique()).reset_index().rename(columns={'index': 'field name'})
non_varying_values_df = num_unique_vals_df[num_unique_vals_df[0] == 1]

#Add column with the non-varying value
col_lst = list(non_varying_values_df['field name'])

col_unique_vals_lst = []

for col in col_lst:
    col_unique_vals_lst.append(df[col].unique().tolist())
    
    
unq_vals_col_df = pd.DataFrame(list(zip(col_lst, col_unique_vals_lst)), columns=['field name', 'unq_val'])
unq_vals_col_df

#Export list of non-varying columns
fullname = os.path.join(outdir, 'nonvarying_cols_list.csv')
unq_vals_col_df.to_csv(fullname, index=False)

# Create general completeness metrics table

In [7]:
#Remove columns (empty, or single values throughout)
null_and_nonvaryingvals_lst = full_null_cols_df['field name'].tolist() + non_varying_values_df['field name'].tolist()
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] #remove unnamed column - redundant index value if exists
df.drop(null_and_nonvaryingvals_lst, axis=1, inplace=True)

In [8]:
#Get number of records in dataframe
num_recs = df.shape[0]
#print(num_recs)

In [9]:
#Create dataframe with number of distinct values in each column
col_lst = list(df.columns)
col_unique_lst = []
for col in col_lst:
    col_unique_lst.append(df[col].nunique())

unq_vals_col_df = pd.DataFrame(list(zip(col_lst, col_unique_lst)), columns=['field name', 'num_unq_vals'])
unq_vals_col_df = unq_vals_col_df.set_index('field name')

In [10]:
#unq_vals_col_df

In [11]:
compl_df = pd.DataFrame((df.isnull().sum(axis = 0)), columns = ['NULL_CNT'])
compl_df['NON_NULL_CNT'] = num_recs-(compl_df['NULL_CNT'])
compl_df['PERC_NULL'] = compl_df['NULL_CNT']/num_recs
compl_df['PERC_NON_NULL'] = compl_df['NON_NULL_CNT']/num_recs
compl_df = compl_df.join(unq_vals_col_df)


#Export list of non-varying columns
fullname = os.path.join(outdir, 'completeness_tabular_results.csv')
compl_df.to_csv(fullname, index=True)

In [12]:
compl_df

,NULL_CNT,NON_NULL_CNT,PERC_NULL,PERC_NON_NULL,num_unq_vals
Id,0,150,0.000000,1.000000,150
SepalLengthCm,0,150,0.000000,1.000000,35
SepalWidthCm,0,150,0.000000,1.000000,23
PetalLengthCm,0,150,0.000000,1.000000,43
PetalWidthCm,0,150,0.000000,1.000000,22
Species,0,150,0.000000,1.000000,3
SepalLengthCm.1,9,141,0.060000,0.940000,34
SepalWidthCm.1,10,140,0.066667,0.933333,23
PetalLengthCm.1,23,127,0.153333,0.846667,43
PetalWidthCm.1,6,144,0.040000,0.960000,22


# Get completeness percent of non-null values. Binned by YY/MM.

In [13]:
#Create new column with extracted year and month
def get_monthly_comp_per(df, field):
    #Create directory if it does not exist
    outdir = './exports/groupby_results'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    #check if field is a date data type field and convert if necessary
    if is_datetime64_any_dtype(df[field]) is False:
        df[field] = pd.to_datetime(df[field])
    
    #create groupby month and year
    res_df = df.groupby([pd.Grouper(key= field, freq="M")], dropna=False).apply(lambda x: x.notnull().mean().round(4))
    res_df = res_df.drop([field], axis=1)
    
    #Export list of non-varying columns
    fullname = os.path.join(outdir, f'completeness_perc_gb_'+ field + '_by_month.csv')
    res_df.to_csv(fullname, index=True)
    
    return(res_df)

In [14]:
#test function
test_df = pd.read_csv("TravelTime_387.csv")
get_monthly_comp_per(test_df, "timestamp") #df name and field name are the parameters

,value,value2,value3
timestamp,,,
2015-07-31,1.0000,0.7531,1.0
2015-08-31,1.0000,0.8437,1.0
2015-09-30,0.9439,1.0000,1.0


# Get completeness percent table based on chosen groupby

In [15]:
#Create function
def get_compl_perc_gb(df, bin_var):
    #Create directory if it does not exist
    outdir = './exports/groupby_results'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    res_df = df.groupby(bin_var, dropna=False).apply(lambda x: x.notnull().mean().round(4))
    res_df = res_df.drop([bin_var], axis=1)
    
    #Export list of non-varying columns
    fullname = os.path.join(outdir, f'completeness_perc_gb_'+ bin_var + '.csv')
    res_df.to_csv(fullname, index=False)
    
    return(res_df)

In [16]:
#Test function
get_compl_perc_gb(df, 'Species') #dataframe name and the categorical column are the parameters

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,SepalLengthCm.1,SepalWidthCm.1,PetalLengthCm.1,PetalWidthCm.1,Species.1
Species,,,,,,,,,,
Iris-setosa,1.0,1.0,1.0,1.0,1.0,0.94,0.96,0.70,0.96,1.0
Iris-versicolor,1.0,1.0,1.0,1.0,1.0,0.92,0.94,0.90,1.00,1.0
Iris-virginica,1.0,1.0,1.0,1.0,1.0,0.96,0.90,0.94,0.92,1.0


# Get completeness non-null count table based on chosen groupby

In [17]:
#Create function
def get_compl_count_gb(df, bin_var):    
    #Create directory if it does not exist
    outdir = './exports/groupby_results'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    res_df =  df.groupby(bin_var).apply(lambda x: x.notnull().sum())
    res_df = res_df.drop([bin_var], axis=1)
    
    #Export list of non-varying columns
    fullname = os.path.join(outdir, f'completeness_cnt_gb_'+ bin_var +'.csv')
    res_df.to_csv(fullname, index=False)
    
    return(res_df)

In [18]:
#Test function
get_compl_count_gb(df, "Species") #dataframe name and the categorical column are the parameters

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,SepalLengthCm.1,SepalWidthCm.1,PetalLengthCm.1,PetalWidthCm.1,Species.1
Species,,,,,,,,,,
Iris-setosa,50,50,50,50,50,47,48,35,48,50
Iris-versicolor,50,50,50,50,50,46,47,45,50,50
Iris-virginica,50,50,50,50,50,48,45,47,46,50


# Get completeness non-null percent time series based on chosen groupby


In [19]:
#Create function
def get_compl_perc_gb_categorical_and_time_series(df, bin_var, time_var):  
    
    #Create directory if it does not exist
    outdir = './exports/groupby_results'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    
    #check if field is a date data type field and convert if necessary
    if is_datetime64_any_dtype(df[time_var]) is False:
        df[time_var] = pd.to_datetime(df[time_var])
    
    #create groupby month and year
    res_df = df.groupby([bin_var, pd.Grouper(key= time_var, freq="M")], dropna=False).apply(lambda x: x.notnull().mean().round(4))
    #res_df = res_df.drop([time_var], axis=1)
    
    #Export list of non-varying columns
    fullname = os.path.join(outdir, f'completeness_cnt_gb_'+ bin_var + '_' + time_var + '.csv')
    res_df.to_csv(fullname, index=False)
    
    return(res_df)

In [20]:
#Test function
test_df = get_compl_perc_gb_categorical_and_time_series(test_df, "value3", "timestamp")

In [21]:
test_df.head(30)

timestamp   value  value2  value3
value3 timestamp                                    
a      2015-07-31        1.0  1.0000  0.7531     1.0
       2015-08-31        1.0  1.0000  0.0000     1.0
b      2015-08-31        1.0  1.0000  0.9052     1.0
       2015-09-30        1.0  1.0000  1.0000     1.0
c      2015-09-30        1.0  1.0000  1.0000     1.0
d      2015-09-30        1.0  0.9204  1.0000     1.0